In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [2]:
browser  = Chrome(executable_path="/Users/juhy9/Downloads/chromedriver_win32/chromedriver.exe")
browser.maximize_window()

In [3]:
def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    
    browser.get(base_url)
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to_frame(browser.find_element_by_id('coinfo_cp'))
    
    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    return pd.DataFrame(td2,columns = col,index = date)

In [8]:
coinfo = stock_crawler("005930")

C:\Anaconda3-32\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.frame instead
  


 기업개요(삼성전자)


In [10]:
coinfo.columns

Index(['매출액', '영업이익', '영업이익(발표기준)', '세전계속사업이익', '당기순이익', '당기순이익(지배)',
       '당기순이익(비지배)', '자산총계', '부채총계', '자본총계', '자본총계(지배)', '자본총계(비지배)', '자본금',
       '영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름', 'CAPEX', 'FCF', '이자발생부채', '영업이익률',
       '순이익률', 'ROE(%)', 'ROA(%)', '부채비율', '자본유보율', 'EPS(원)', 'PER(배)',
       'BPS(원)', 'PBR(배)', '현금DPS(원)', '현금배당수익률', '현금배당성향(%)', '발행주식수(보통주)'],
      dtype='object')

In [11]:
coinfo

,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,...,부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
2015/12,"2,006,535","264,134","264,134","259,610","190,601","186,946","3,655","2,421,795","631,197","1,790,598",...,35.25,"21,117.88","2,198",11.47,"23,715",1.06,420,1.67,16.42,"7,364,966,850"
2016/12,"2,018,667","292,407","292,407","307,137","227,261","224,157","3,104","2,621,743","692,113","1,929,630",...,35.87,"22,004.14","2,735",13.18,"26,636",1.35,570,1.58,17.81,"7,033,966,850"
2017/12,"2,395,754","536,450","536,450","561,960","421,867","413,446","8,422","3,017,521","872,607","2,144,914",...,40.68,"24,536.12","5,421",9.40,"30,427",1.67,850,1.67,14.09,"6,454,924,700"
2018/12,"2,437,714","588,867","588,867","611,600","443,449","438,909","4,540","3,393,572","916,041","2,477,532",...,36.97,"27,531.92","6,024",6.42,"35,342",1.09,"1,416",3.66,21.92,"5,969,782,550"
2019/12,"2,304,009","277,685","277,685","304,322","217,389","215,051","2,338","3,525,645","896,841","2,628,804",...,34.12,"28,856.02","3,166",17.63,"37,528",1.49,"1,416",2.54,44.73,"5,969,782,550"
2020/12,"2,302,575","320,365",0,"345,236","249,827","247,232",0,"3,705,026","919,868","2,785,158",...,33.03,0,"3,640",14.53,"39,777",1.33,"1,525",2.88,36.83,0
2021/12,"2,573,140","453,591",0,"482,084","349,398","345,558",0,"3,990,757","967,894","3,022,864",...,32.02,0,"5,087",10.40,"43,227",1.22,"1,599",3.02,27.63,0
2022/12,"2,806,204","515,397",0,"548,693","398,325","394,787",0,"4,346,441","1,044,211","3,302,230",...,31.62,0,"5,812",9.10,"47,298",1.12,"1,596",3.02,24.14,0


In [ ]:
stock_crawler("023530")

In [ ]:
stock_crawler("005930")

In [ ]:
stock_crawler("000660")

In [6]:
import pandas as pd
import requests

URL = "https://finance.naver.com/item/main.nhn?code=005930"

samsung_electronic = requests.get(URL)
html = samsung_electronic.text

financial_stmt = pd.read_html(samsung_electronic.text)[3]

financial_stmt.set_index(('주요재무정보', '주요재무정보', '주요재무정보'), inplace=True)
financial_stmt.index.rename('주요재무정보', inplace=True)
financial_stmt.columns = financial_stmt.columns.droplevel(2)

In [7]:
financial_stmt

최근 연간 실적                                       최근 분기 실적  \
              2017.12     2018.12     2019.12  2020.12(E)    2019.03   
주요재무정보                                                                 
매출액        2395754.00  2437714.00  2304009.00  2302575.00  523855.00   
영업이익        536450.00   588867.00   277685.00   320365.00   62333.00   
당기순이익       421867.00   443449.00   217389.00   249827.00   50436.00   
영업이익률           22.39       24.16       12.05       13.91      11.90   
순이익률            17.61       18.19        9.44       10.85       9.63   
ROE(지배주주)       21.01       19.63        8.69        9.42      16.21   
부채비율            40.68       36.97       34.12         NaN      36.27   
당좌비율           181.61      204.12      233.57         NaN     209.25   
유보율          24536.12    27531.92    28856.02         NaN   27832.98   
EPS(원)        5421.00     6024.00     3166.00     3640.00     752.00   
PER(배)           9.40        6.42       17.63       14.53       8.54   
BPS(원)       30427.00    35342.00    37528.00    39777.00   36142.00   
PBR(배)           1.67        1.09        1.49        1.33       1.24   
주당배당금(원)       850.00     1416.00     1416.00     1525.00        NaN   
시가배당률(%)         1.67        3.66        2.54         NaN        NaN   
배당성향(%)         14.09       21.92       44.73         NaN        NaN   

                                                                  
             2019.06    2019.09    2019.12    2020.03 2020.06(E)  
주요재무정보                                                            
매출액        561271.00  620035.00  598848.00  553252.00  503521.00  
영업이익        65971.00   77779.00   71603.00   64473.00   61314.00  
당기순이익       51806.00   62877.00   52270.00   48849.00   50847.00  
영업이익률          11.75      12.54      11.96      11.65      12.18  
순이익률            9.23      10.14       8.73       8.83      10.10  
ROE(지배주주)      13.23      10.05       8.69       8.45        NaN  
부채비율           33.05      34.14      34.12      34.19        NaN  
당좌비율          230.74     235.80     233.57     237.80        NaN  
유보율         28129.35   28541.64   28856.02   29134.12        NaN  
EPS(원)        746.00     899.00     770.00     720.00     699.00  
PER(배)         10.48      13.73      17.63      15.24      75.68  
BPS(원)      36789.00   37600.00   37528.00   38053.00        NaN  
PBR(배)          1.28       1.30       1.49       1.25        NaN  
주당배당금(원)         NaN        NaN        NaN        NaN        NaN  
시가배당률(%)         NaN        NaN        NaN        NaN        NaN  
배당성향(%)          NaN        NaN        NaN        NaN        NaN

In [12]:

import urllib.request
from bs4 import BeautifulSoup
code =['005930']
for c in code:
    url="http://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A" + c + "&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701"
    f = urllib.request.urlopen(url).read()
    soup=BeautifulSoup(f, 'html.parser')
    cells = soup.find('tr', {'id': "p_grid1_10"}).find_all('td')
    for cell in cells:
        print (cell.string)

6.85
6.12
4.01
7.42
 
